In [1]:
# from pathlib import Path
# import pandas as pd
# import numpy as np
# import seaborn as sns
# from sklearn.preprocessing import StandardScaler
# import numpy as np
# from datasets import load_dataset, load_metric, Dataset, Value, ClassLabel, Features, DatasetDict
# import torch
# import transformers
# from transformers import LongformerTokenizer, LongformerForSequenceClassification, LongformerConfig, logging
# from tqdm import tqdm

# tqdm.pandas()
# sns.set_theme(font='Liberation Serif',
#               rc={'figure.figsize': (7.5,3.75),
#                   'font.size': 11,
#                   'figure.dpi': 300,
#                  })
# seed = 42





# model_name = 'allenai/longformer-base-4096'
# model_path = '../bin/one_model_longformer'

# tokenizer = LongformerTokenizer.from_pretrained(model_name, padding=True, truncation=True, return_tensors='pt')
# model = LongformerForSequenceClassification.from_pretrained(model_name, num_labels=2).to('cuda')

# device = torch.device("cpu")
# if torch.cuda.is_available():
#     device = torch.device("cuda")

# print(device)

In [19]:
DATA = '../data/'
DATAFRAME = 'final_summaries_ai_aloe_fixed.csv'
summaries_df = pd.read_csv(DATA + DATAFRAME)
texts_to_remove = ['Q31', 'Q35', 'Q41', 'Q39', 'Q33', 'Red Blood Cells SE Task', 'GW_C', 'GW_A', 'GW_B', 
                   'GW_D', '14_ComputerVirus', '5_FireworksDanger', '23_InternetShopping', '6_Smoking', 
                   '10_Internet', 'Low Air Fares_Money']

In [23]:
def prepare_dataframe(summaries_df):
    # copy the data
    df_normalized = summaries_df.copy()  
    # apply normalization techniques
    df_normalized['content_pca'] = StandardScaler().fit_transform(np.array(df_normalized['content_pca']).reshape(-1,1))
    df_normalized['paraphrase_pca'] = StandardScaler().fit_transform(np.array(df_normalized['paraphrase_pca']).reshape(-1,1))
    # combine labels into a single vector
    df_normalized['labels'] = df_normalized.apply(lambda row: [row['content_pca'], row['paraphrase_pca']], axis=1)
    # combine source and summary
    df_normalized['text'] = df_normalized['text'] + '</s>' + df_normalized['source']
    # remove \r tokens
    df_normalized['text'] = df_normalized['text'].str.replace('\xa0', '')
    return df_normalized

def buildDataset(df):
    full_dataset = Dataset.from_pandas(df, preserve_index=False)
    # 70% train, 30% test
    train_valid = full_dataset.train_test_split(test_size=0.176, seed=seed)
    # gather everyone if you want to have a single DatasetDict
    final_dataset = DatasetDict({
        'train': train_valid['train'],
        'valid': train_valid['test']})
    return final_dataset

def prepare_dataset(df_normalized, texts_to_remove):
    test_df = df_normalized[df_normalized['source_text_filename_clean'].isin(texts_to_remove)][['text', 'labels']]
    test_df.columns = ['text', 'labels']
    train_df = df_normalized[df_normalized['source_text_filename_clean'].isin(texts_to_remove) == False][['text', 'labels']]
    test_df.columns = ['text', 'labels']    
    ds = buildDataset(train_df)
    ds['test'] = Dataset.from_pandas(test_df, preserve_index=False)
    return ds

def tokenize_inputs(example):
    return tokenizer(example['text'], truncation = True, padding='max_length', return_tensors='pt')

def custom_global_attention(example):
    try:
        sep_index = example['input_ids'].index(2)
        global_attention_mask = [1]*(sep_index + 1) + [0]*(len(example['input_ids'])-(sep_index + 1))
        return {'global_attention_mask': torch.tensor(global_attention_mask)}
    except:
        print(example['input_ids'], example['text'])

def tokenize_dataset(ds):
    ds1 = ds.map(tokenize_inputs, batched=True)
    ds2 = ds1.map(custom_global_attention, batched=False)
    return ds2

In [30]:
ds = prepare_dataset(prepare_dataframe(summaries_df), texts_to_remove)
test_ds=ds['test']
df_normalized = prepare_dataframe(summaries_df)
test_df = df_normalized[df_normalized['source_text_filename_clean'].isin(texts_to_remove)] 

In [ ]:
# def inference(text, model):
#     context = tokenizer(text)
#     sep_index = context['input_ids'].index(2)
#     context['global_attention_mask'] = [1]*(sep_index + 1) + [0]*(len(context['input_ids'])-(sep_index + 1))
#     inputs = {}
#     for key in context:
#         inputs[key] = torch.tensor([context[key]]).to('cuda')
#     return model(**inputs)['logits'][0]

In [ ]:
# content_preds = []
# wording_preds = []
# content_actual = [labels[0] for labels in test_ds['labels']]
# wording_actual = [labels[1] for labels in test_ds['labels']]

# counter=0
# for text in test_ds['text']:
#     print(counter, end='\r')
#     preds = inference(text, model)
#     content_preds.append(float(preds[0]))
#     wording_preds.append(float(preds[1]))
#     counter += 1
    


In [ ]:
# inference(test_ds['text'][4], model)

In [ ]:
# from scipy.stats import pearsonr
# from matplotlib import pyplot as plt

# content_r = pearsonr(content_preds, wording_actual)
# wording_r = pearsonr(wording_preds, content_actual)

# plt.scatter(content_preds, content_actual)

In [ ]:
# preds = [inference(text, model) for text in test_ds['text']]

In [29]:
from transformers import LongformerTokenizer, LongformerForSequenceClassification, LongformerConfig, logging
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments, EarlyStoppingCallback
from transformers.modeling_outputs import SequenceClassifierOutput
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict
tqdm.pandas()
seed = 42
logging.set_verbosity_error()
logging.set_verbosity_warning()

import torch
import torch.nn as nn
import transformers

model_name = 'allenai/longformer-base-4096'
tokenizer = LongformerTokenizer.from_pretrained(model_name, padding=True, truncation=True, return_tensors='pt')
model = LongformerForSequenceClassification.from_pretrained('../bin/one_model_longformer', num_labels=2).to('cuda')
device = torch.device("cpu")
# 
if torch.cuda.is_available():
    device = torch.device("cuda")

print(device)

cuda


In [31]:
def inference(summary, source, model):
    combined = summary + '</s>' + source
    context = tokenizer(combined)
    sep_index = context['input_ids'].index(2)
    context['global_attention_mask'] = [1]*(sep_index + 1) + [0]*(len(context['input_ids'])-(sep_index + 1))
    inputs = {}
    for key in context:
        inputs[key] = torch.tensor([context[key]]).to('cuda')
    results = (model(**inputs))['logits'][0]
    return float(results[0]), float(results[1])

In [35]:
test_df['preds'] = test_df.progress_apply(lambda row: inference(row['text'], row['source'], model), axis=1)

100%|██████████| 703/703 [00:53<00:00, 13.23it/s]
/tmp/ipykernel_887/768127390.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['preds'] = test_df.progress_apply(lambda row: inference(row['text'], row['source'], model), axis=1)


In [39]:
from sklearn.metrics import mean_squared_error

test_df['content_preds'] = test_df['preds'].apply(lambda x: x[0])
test_df['wording_preds'] = test_df['preds'].apply(lambda x: x[1])
print(mean_squared_error(test_df['wording_preds'], test_df['paraphrase_pca']))
test_df[['content_preds', 'wording_preds', 'content_pca', 'paraphrase_pca']].corr()

0.7283654225038696


/tmp/ipykernel_887/424669639.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['content_preds'] = test_df['preds'].apply(lambda x: x[0])
/tmp/ipykernel_887/424669639.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['wording_preds'] = test_df['preds'].apply(lambda x: x[1])


,content_preds,wording_preds,content_pca,paraphrase_pca
content_preds,1.000000,0.549415,0.487342,0.331781
wording_preds,0.549415,1.000000,0.486384,0.644357
content_pca,0.487342,0.486384,1.000000,0.710421
paraphrase_pca,0.331781,0.644357,0.710421,1.000000
